<a href="https://colab.research.google.com/github/tomonari-masada/course2025-nlp/blob/main/10_LLM_finetuning_for_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 言語モデルを使ったテキスト分類
* 今回はトランスフォーマ言語モデルのファインチューニングを実践する。
  * パラメータ数は数億個オーダのもの。
  * パラメータ数が数十億個（数ビリオン）のものは、扱いがやや大変。
* ファインチューニングによってテキスト分類の性能を向上させる。

* 今回はTransformersライブラリを使う。
  * Sentence Transformersは使わない。

* ランタイムのタイプをGPUに設定しておく。

## 準備

In [ ]:
!pip install -U evaluate

In [ ]:
from tqdm.auto import tqdm
import numpy as np
import torch
from datasets import load_from_disk
import evaluate
from transformers import set_seed, AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

set_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## データセット
* ライブドアニュースコーパスを使う。
* 前々回に作成したtraining/validation/testのsliceを使う。
  * https://github.com/tomonari-masada/course2025-nlp/blob/main/livedoor_ds.tar.gz

In [ ]:
!wget https://github.com/tomonari-masada/course2025-nlp/raw/refs/heads/main/livedoor_ds.tar.gz
!tar zxf livedoor_ds.tar.gz

In [ ]:
ds = load_from_disk("livedoor_ds")
ds

In [ ]:
set(ds["train"]["category"])

In [ ]:
category_names = [
  'movie-enter',
  'it-life-hack',
  'kaden-channel',
  'topic-news',
  'livedoor-homme',
  'peachy',
  'sports-watch',
  'dokujo-tsushin',
  'smax',
]

num_labels = len(set(ds["train"]["category"]))
num_labels

In [ ]:
ds["train"]["content"][0]

## トークナイザ

* E5の多言語版を使う。
  * https://arxiv.org/abs/2212.03533
  * https://arxiv.org/abs/2402.05672

* テキストのトークン化をここで済ませておく。
  * トークン化は、training/validation/test setの分割とは無関係の処理。
  * だから、最初にデータセット全体をトークン化しておいても、問題はない。

In [ ]:
model_name = "intfloat/multilingual-e5-large-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(examples):
    return tokenizer(
        examples["content"],
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )

tokenized_ds = ds.map(preprocess, batched=True)

* 正解ラベルのカラムを"label"にrenameする。

In [ ]:
for slice in tokenized_ds:
    tokenized_ds[slice] = tokenized_ds[slice].rename_column("category", "label")

In [ ]:
train_ds = tokenized_ds["train"]
eval_ds = tokenized_ds["validation"]

In [ ]:
train_ds

## 言語モデル

* `AutoModelForSequenceClassification`クラスを使う。

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels, # ここで分類クラス数を指定
).to(device)

* モデルの中身を見てみる。
  * `classifier`というモジュールに注目。

In [ ]:
model

In [ ]:
model.classifier

In [ ]:
model.classifier.dense.weight.shape

In [ ]:
model.classifier.out_proj.weight.shape

* 分類器として使えることを確認する。

In [ ]:
input = preprocess(tokenized_ds["test"][0]).to(model.device)
input

In [ ]:
model.eval()
with torch.no_grad():
    logits = model(**input).logits
model.train()
logits

In [ ]:
torch.argmax(model(**input).logits, axis=-1)

* ファインチューニング前のモデルのtest set上での分類性能を見てみる。
  * 当然、ランダムな予測の分類性能に近い。
  * 分類用のヘッドが全くtrainingされていないから。

In [ ]:
def evaluate_model(model, ds, eval_batch_size=1):
    model.eval()
    predicted_class_ids = []
    offset = 0
    for offset in tqdm(range(0, len(ds), eval_batch_size)):
        examples = ds[offset:offset + eval_batch_size]
        input = preprocess(examples).to(model.device)
        with torch.no_grad():
            logits = model(**input).logits
        predicted_class_ids.append(torch.argmax(logits, axis=-1))
    model.train()
    return torch.concat(predicted_class_ids)

predicted_class_ids = evaluate_model(model, ds["test"], eval_batch_size=32)

In [ ]:
print(predicted_class_ids)

* evaluateライブラリを使って評価する。

In [ ]:
metric = evaluate.load("accuracy")
metric.compute(predictions=predicted_class_ids, references=ds["test"][:]["category"])

## 評価を実行するヘルパ関数

* logitsと正解ラベルを渡せばaccuracyを返してくれる関数。

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

## `Trainer`の作成

In [ ]:
training_args = TrainingArguments(
    output_dir="my_model",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    eval_strategy="steps",
    eval_steps=50,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    compute_metrics=compute_metrics,
)

* モデルの重みを部分的に更新したい場合は、以下のようにする。
  * これは、classifier headと最後の4層だけをtrainingする例。
  * このぐらいのパラメータ数の埋め込みモデルであれば、full finetuningする方が良い。

In [ ]:
#for name, param in model.named_parameters():
#  if "classifier" in name:
#    param.requires_grad = True
#  elif ("roberta.encoder.layer.2" in name
#        and "roberta.encoder.layer.2." not in name):
#    param.requires_grad = True
#  else:
#    param.requires_grad = False

* 以下は、強制的に、全パラメータを更新するように設定するコード。
  * 大体のモデルは、デフォルトで全てのパラメータの`requires_grad`は`True`になっているはず。

In [ ]:
for param in model.parameters():
    param.requires_grad = True

## ファインチューニングの実行

* CUDA out of memoryエラーが出たら・・・
  * バッチサイズを小さくするなど、GPUのメモリを使わない工夫をしてから・・・
  * セッションを再起動して最初のセルからやり直す。


In [ ]:
trainer.train()

In [ ]:
trainer.save_model(f"my_model")

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("my_model").to(device)

In [ ]:
predicted_class_ids = evaluate_model(model, ds["test"], eval_batch_size=32)

In [ ]:
metric.compute(predictions=predicted_class_ids, references=ds["test"][:]["category"])